In [18]:
import random

# Constants
STAIRCASE_LENGTH = 10  # Number of steps in the staircase
K = 4  # Distance within which agents can see each other

# Directions
UP = -1
DOWN = 1

# Slot indices
LEFT = 0
MIDDLE = 1
RIGHT = 2

# Flag
RETURN_TO_MIDDLE = False



# Initialize the staircase
# Each step is represented as a list of 3 slots: [left, middle, right]
# None means the slot is empty, otherwise it stores the direction of the agent (UP or DOWN)
staircase = [[None, None, None] for _ in range(STAIRCASE_LENGTH)]

# Initialize step counts
# Each step is represented as a list of 3 slots: [left, middle, right]
# Each slot stores the cumulative number of timesteps agents have spent there
step_counts = [[0, 0, 0] for _ in range(STAIRCASE_LENGTH)]

def print_staircase():
    """Print the current state of the staircase."""
    for step in staircase:
        print("|".join([" " if slot is None else "↑" if slot == UP else "↓" for slot in step]))
    print("-" * (3 * STAIRCASE_LENGTH))

# def print_step_counts():
#     """Print the cumulative step counts for each slot in the staircase."""
#     print("Cumulative Step Counts:")
#     for step in step_counts:
#         print("|".join([f"{count:3}" for count in step]))
#     print("-" * (3 * STAIRCASE_LENGTH))

def print_step_counts():
    """Print the cumulative step counts for each slot in the staircase as a Python array."""
    print("step_counts = [")
    for i, step in enumerate(step_counts):
        # Format the step as a list of integers
        step_str = "    [" + ", ".join(f"{count:3}" for count in step) + "]"
        # Add a comma unless it's the last step
        if i < STAIRCASE_LENGTH - 1:
            step_str += ","
        print(step_str)
    print("]")
    return f'[{step_str}]'

def spawn_agent(position, direction):
    """Spawn an agent at the given position and direction."""
    step = staircase[position]

    for d in range(1, K + 1):
        if 0 <= position + d * direction < STAIRCASE_LENGTH:  # scan ahead
            opposing_step = staircase[position + d * direction]
            if any(slot == -direction for slot in opposing_step):
                # Move to the rightmost slot from their perspective
                if direction == DOWN:
                    step[LEFT] = direction
                else:
                    step[RIGHT] = direction
                break
    else:
        step[MIDDLE] = direction

def move_agents():
    """Move all agents according to the rules."""
    global staircase
    new_staircase = [[None, None, None] for _ in range(STAIRCASE_LENGTH)]

    for i in range(STAIRCASE_LENGTH):
        for j in range(3):
            if staircase[i][j] is not None:
                direction = staircase[i][j]
                new_position = i + direction

                ### Agent remains on staircase
                if 0 <= new_position < STAIRCASE_LENGTH:
                    # Check if the agent needs to move to the rightmost slot
                    if j == MIDDLE:
                        # Check for opposing agents within distance K
                        for d in range(1, K + 1):
                            if 0 <= i + d * direction < STAIRCASE_LENGTH:  # scan ahead
                                opposing_step = staircase[i + d * direction]
                                if any(slot == -direction for slot in opposing_step):
                                    # Move to the rightmost slot from their perspective
                                    if direction == DOWN:
                                        target_slot = LEFT
                                    else:
                                        target_slot = RIGHT
                                    break
                        else:  # no opposing people in sight
                            target_slot = j

                    else:  # is not in the middle
                        if RETURN_TO_MIDDLE:
                            for d in range(1, K + 1):
                                if 0 <= i + d * direction < STAIRCASE_LENGTH:  # scan ahead
                                    opposing_step = staircase[i + d * direction]
                                    if any(slot == -direction for slot in opposing_step):
                                        # Move to the rightmost slot from their perspective
                                        if direction == DOWN:
                                            target_slot = LEFT
                                        else:
                                            target_slot = RIGHT
                                        break
                            else:  # no opposing people in sight
                                target_slot = MIDDLE
                        else:
                            target_slot = j

                    # Move the agent
                    new_staircase[new_position][target_slot] = direction

                else:
                    # Agent has reached the end of the staircase and is removed
                    pass

    # Update step counts for the current timestep
    for i in range(STAIRCASE_LENGTH):
        for j in range(3):
            if staircase[i][j] is not None:
                step_counts[i][j] += 1

    staircase = new_staircase

def simulate(steps):
    """Run the simulation for a given number of steps."""
    for step in range(steps):
        # print(f"Step {step + 1}:")
        # Spawn new agents at the top and bottom


        if random.random() < 0.03:
            spawn_agent(0, DOWN)
        if random.random() < 0.02:
            spawn_agent(STAIRCASE_LENGTH - 1, UP)



        # print_staircase()
        move_agents()

    # Print cumulative step counts at the end of the simulation
    return eval(print_step_counts())

# Run the simulation for 10 steps
data = simulate(5000)

step_counts = [
    [ 14, 199,  46],
    [ 14, 202,  45],
    [ 20, 198,  43],
    [ 23, 197,  41],
    [ 30, 196,  35],
    [ 39, 188,  34],
    [ 44, 188,  29],
    [ 49, 190,  22],
    [ 52, 194,  15],
    [ 52, 194,  15]
]


In [19]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.cm import ScalarMappable
from matplotlib.colors import Normalize

def create_used_stairs_3d_histogram_with_heatmap(N=10, steps_data=None, wear_factor=0.01):
    if steps_data is None:
        steps_data = np.random.randint(0, 100, size=(N, 3))  

    default_heights = np.arange(1, N + 1).reshape(-1, 1) 

    used_heights = default_heights - wear_factor * steps_data

    x = np.arange(3)  # Tiles: 0 (left), 1 (center), 2 (right)
    y = np.arange(N)  # Stairs: 0 to N-1
    x, y = np.meshgrid(x, y)

    x = x.flatten()
    y = y.flatten()
    z = np.zeros_like(x)  
    dx = dy = 1.0  
    dz = used_heights.flatten()  

    wear_values = (wear_factor * steps_data).flatten()  

    norm = Normalize(vmin=np.min(wear_values), vmax=np.max(wear_values))
    cmap = plt.get_cmap('viridis_r')  
    colors = cmap(norm(wear_values))  

    fig = plt.figure(figsize=(10, 8))#figsize=(20, 8))

    ax1 = fig.add_subplot(111, projection='3d')
    ax1.bar3d(x, y, z, dx, dy, dz, shade=True, color=colors)
    ax1.grid(False)
    ax1.set_axis_off()
    ax1.set_box_aspect([3, N, np.max(used_heights)]) 
    ax1.view_init(elev=7, azim=-110, roll=3)

    # sm = ScalarMappable(norm=norm, cmap=cmap)
    # sm.set_array(wear_values)
    # cbar1 = plt.colorbar(sm, ax=ax1, shrink=0.5, aspect=10)
    # cbar1.set_label('Wear Effect')

    # ax2 = fig.add_subplot(122)
    # heatmap = ax2.imshow(steps_data, cmap='viridis_r', aspect='auto', origin='lower')
    # ax2.set_xticks(np.arange(3))
    # ax2.set_xticklabels(['Left', 'Center', 'Right'])
    # ax2.set_yticks(np.arange(N))
    # ax2.set_yticklabels([f'Step {i+1}' for i in range(N)])
    # ax2.set_xlabel('Tile')
    # ax2.set_ylabel('Step')
    # ax2.set_title('Number of Activations per Tile')

    # cbar2 = plt.colorbar(heatmap, ax=ax2, shrink=0.5, aspect=10)
    # cbar2.set_label('Number of Activations')

    plt.savefig("high_res_plot.png", dpi=300, bbox_inches='tight')
    plt.tight_layout()
    plt.show()
    

N = 10  

# steps_data = np.array([
#     [358, 245, 171],
#     [358, 174, 253],
#     [359, 117, 309],
#     [365,  80, 340],
#     [357,  92, 335],
#     [355,  93, 335],
#     [353,  97, 333],
#     [315, 124, 345],
#     [271, 166, 347],
#     [159, 271, 347]
# ])

steps_data = np.array(data)



create_used_stairs_3d_histogram_with_heatmap(N, steps_data, wear_factor=0.0005)

TypeError: unsupported operand type(s) for *: 'float' and 'NoneType'